# Pythonでゴブレットゴブラーズ(簡易)の解析

- [ゴブレットゴブラーズ (日本語版)](https://sgrk.blog.fc2.com/blog-entry-3687.html)
- [ソースコード設計スライド](https://docs.google.com/presentation/d/1K8fbRlk24Y_J3M--F-WnYYN0hddL89w3EGN4Zgw-L1g/edit?usp=sharing)

- [卒論概要](https://docs.google.com/document/d/1jEXvVmpUPCaowl3xR1qgyhUFGVHFVzyZjBo9F8OAR7M/edit?usp=sharing)

## 1. 設計

### a) state

必要な情報を60bitで表す。  
state = 0b_0_0_0_000000_000000000_000000000000000000000000000000000000000000000000000000_000000000000


| 区分 | is_first | is_choise | is_choise_board | choise_piece_type | choise_piece | board | hand |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| bit数 | 1bit | 1bit | 1bit | 4bit | 9bit | 36bit | 8bit |
| 例 | 1 | 1 | 1 | 1000 | 100000000 | 010000000 001000000 001010000 000000000 | 00 01 00 11 |
| 説明 | 手番 | モード | 持ち上げている駒は盤か手か | 持ち上げている駒の種類[rl,rm,bl,bm] | 持ち上げている駒のマス[0,1,2,3,4,5,6,7,8] | 盤面に置いてある駒[rl0, rl1, ..., rm0..., bl0...] | 手駒[rlの個数, rmの...] |

説明の補足 :   
- hand  “00”...その駒は持ってない, “01”...一つ持ってる, “11”...２つ持ってる

- stateは盤面に駒を配置したset_stateと行動する駒を選択したchoise_stateの２種類に分けられる。
set_stateは
```
choise_piece_type == 0b000000000
```
であり、choise_stateでは何らかのbitが立っている。
- choise_stateでしかis_choise, is_choise_board, choise_piece_typeはbitが立たない。
- choise_stateで選択された駒はboard, handからは除外される。（浮いている駒）

確認

In [31]:
# 確認
state = 0b_1_0_0_1000_100000000_100000000000000000000000000000000000_10000000
print(state) # 10進数
print(f'{state:060b}')#bin(プレフィックスなし)
print(type(f'{state:060b}')) #文字列です
print(len(f'{state:060b}')) # bit数の確認

653030742061744256
100100010000000010000000000000000000000000000000000010000000
<class 'str'>
60


bit操作

In [32]:
# 1bit取り出す
print(f'{state:084b}'[0])
# 複数のbitを取り出す
print(f'{state:060b}'[1+1+1:1+1+1+4])
# 1bit書き換え
print("変更前 : " + f'{state:060b}')
state = int(f'{state:060b}'[:1] + '1' + f'{state:060b}'[2:], 2)
print("変更後 : " + f'{state:060b}')
# 複数のbitを書き換え
print("変更前 : " + f'{state:060b}')
state = int(f'{state:060b}'[:2] + '1' + f'{state:060b}'[3:52] + '11110000', 2)
print("変更後 : " + f'{state:060b}')

0
1000
変更前 : 100100010000000010000000000000000000000000000000000010000000
変更後 : 110100010000000010000000000000000000000000000000000010000000
変更前 : 110100010000000010000000000000000000000000000000000010000000
変更後 : 111100010000000010000000000000000000000000000000000011110000


複数bitの取り出しチャンク

In [33]:
# バラす
is_first = f'{state:060b}'[0]
is_choise = f'{state:060b}'[1]
is_choise_board = f'{state:060b}'[2]
choise_piece_type = f'{state:060b}'[3:3+4]
choise_piece = f'{state:060b}'[7:7+9]
board = f'{state:060b}'[16:16+36]
hand = f'{state:060b}'[52:52+8]
print("バラす  : " + is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand)

バラす  : 111100010000000010000000000000000000000000000000000011110000


### b) action

set_action, choise_actionともに、行動する駒を表す。

必要な情報を14bitで表す。  
action = 0b_0_0000_000000000


| 区分 | is_choise_board | choise_piece_type | choise_place |
| ---- | ---- | ---- | ---- | 
| bit数 | 1bit | 4bit | 9bit | 
| 例 | 1 | 1000 | 100000000 | 
| 説明 | 行動する駒は盤か手か | 行動する駒の種類[rl,rm,bl,bm] | 行動する駒のマス[0,1,2,3,4,5,6,7,8] | 

説明の補足 : 
- set_actionは選択してある駒を配置する行動なので、この行動によりstateはchoise_stateからset_stateへと変化する。
- choise_actionは駒を選択する行動なので、この行動によりstateはset_stateからchoise_stateへと変化する。

確認

In [34]:
# 確認
action = 0b_1_1000_100000000
print(action) # 10進数
print(f'{action:014b}')#bin(プレフィックスなし)
print(type(f'{action:014b}')) #文字列です
print(len(f'{action:014b}')) # bit数の確認

12544
11000100000000
<class 'str'>
14


bit操作

In [35]:
# 1bit取り出す
print(f'{action:014b}'[0])
# 複数のbitを取り出す
print(f'{action:014b}'[1:1+4])
# 書き換え
print("変更前 : " + f'{action:014b}')
action = int(f'{action:014b}'[:1] + '0001' + '000000010', 2)
print("変更後 : " + f'{action:014b}')

1
1000
変更前 : 11000100000000
変更後 : 10001000000010


複数bitの取り出しチャンク

In [36]:
# バラす
is_choise_board = f'{action:014b}'[0]
choise_piece_type = f'{action:014b}'[1:1+4]
choise_piece = f'{action:014b}'[5:5+9]
print("バラす  : " + is_choise_board + choise_piece_type + choise_piece)

バラす  : 10001000000010


## 関数宣言

### 正規化

盤面を回転、線対称、点対称させることで、同意の状態を一つの状態として扱う。  
変換後、board部が最も小さい状態で正規化する。

In [78]:
import numpy as np

# 変換表を定義( 何度も定義しないように関数外で宣言)
convert_tables = np.array(
    [[6,3,0,7,4,1,8,5,2],[8,7,6,5,4,3,2,1,0],
     [2,5,8,1,4,7,0,3,6],[2,1,0,5,4,3,8,7,6],
     [6,7,8,3,4,5,0,1,2],[0,3,6,1,4,7,2,5,8],[8,5,2,7,4,1,6,3,0]])

def convert_normalization_state(state):
  is_first = f'{state:060b}'[0]
  is_choise = f'{state:060b}'[1]
  is_choise_board = f'{state:060b}'[2]
  choise_piece_type = f'{state:060b}'[3:3+4]
  choise_piece = f'{state:060b}'[7:7+9]
  board = f'{state:060b}'[16:16+36]
  hand = f'{state:060b}'[52:52+8]

  # 変換表でboardを変換したcand_boardを作成し、boardと比較して小さいなら上書きする
  # もしより小さい候補が見つかったら、それと同じ変換表でchoise_pieceも変換する
  board_list = np.array(list(board))
  choise_piece_list = np.array(list(choise_piece))
  for convert_table in convert_tables:
    #     cand_board_list = [0] * 36
    cand_board_list = np.empty(36, dtype = str)
    for i in range(4):
      for after_num, before_num in enumerate(convert_table):
        cand_board_list[i*9 + after_num] = board_list[i*9 + before_num]
    # print("cand_board : " + "".join(cand_board_list))  # デバッグ用
    # board_list と cand_board_list を比較
    board = board_list.tolist()
    cand_board = cand_board_list.tolist()
    if board > cand_board:
      board_list = cand_board_list # 上書き
      for after_num, before_num in enumerate(convert_table): # choise_pieceの変換
        choise_piece_list[after_num] = choise_piece_list[before_num]

  # board_list, choise_piece_listをboard, choise_pieceに変換
  board = "".join(board_list.tolist())
  choise_piece = "".join(choise_piece_list.tolist())

  normalization_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return normalization_state

確認

In [79]:
# 確認
# 先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
state = 0b_1_0_0_0000_000000000_100000000_000000000_000000000_000000000_01111111
print("先手L駒0マスにおく -> 先手L駒は8マスに")
print(f'結果 : normalization_state = {convert_normalization_state(state):060b}')

先手L駒0マスにおく -> 先手L駒は8マスに
結果 : normalization_state = 100000000000000000000000100000000000000000000000000001111111


### 合法手

stateからその合法手一覧であるactionsを作成する

actions:
```
actions = [action1, action2, ...]
```

stateの種類ごとに作成:
- choise_stateから合法手を作成 : leagal_actions_from_choise_state(choise_state)
- set_stateから合法手を作成 : leagal_actions_from_set_state(set_state)

#### leagal_actions_from_choise_state(choise_state) -> set_actions:  
setする駒と場所一覧... set_actions を作成する  
setできる駒と場所とは....
- handは行動できない。あくまで選んだ駒を盤面に配置するという作業だから。
- choiseしている駒と同じ駒しか行動できない。例: rlを選んだならrlしか行動できない。
- choiseした駒以上の大きさの駒があるマスには行動できない。
- choiseした駒が盤面なら、
    - choiseした駒があったマスには行動できない。

In [80]:
# choise_stateから合法手を作成
#
# 出力するactionはすべてset_actionである。

def leagal_actions_from_choise_state(choise_state):
  is_choise_board = f'{choise_state:060b}'[2]
  choise_piece_type = f'{choise_state:060b}'[3:3+4]
  choise_piece = f'{choise_state:060b}'[7:7+9]
  board = f'{choise_state:060b}'[16:16+36]  

  # set_actions_bitを作成
  set_actions_bit = f'{0:036b}' # 36bit。すべての駒と位置が行動不能で初期化
  # choiseしている駒と同じ駒は行動できる。
  cpy_f = choise_piece_type.find('1')
  set_actions_bit = set_actions_bit[:9*cpy_f] + '111111111' + set_actions_bit[9*(cpy_f+1):]
  # choiseしている駒がboardの駒なら、そのマスに行動出来ない
  if is_choise_board == '1': 
    cp_f = choise_piece.find('1')
    set_actions_bit = set_actions_bit[:9*cpy_f+cp_f] + '0' + set_actions_bit[9*cpy_f+cp_f+1:]
  # choiseした駒以上の大きさの駒があるマスには行動できない。
  for mass in range(9):
    if board[mass] == '1' or board[mass+18] == "1": # L駒があるか
      set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]
    if cpy_f in [1,3]: #選択してあるこまがMなら
      if board[mass+9] == '1' or board[mass+18+9] == "1": # M駒があるか
        set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]

  # actions_bitをactionsに変換
  set_actions = []
  for i, bit in enumerate(set_actions_bit):
    if bit == '1':
      set_action = int('0b_1_0000_000000000', 2)
      set_action = int(f'{set_action:014b}'[:1+(i//9)] + '1' + f'{set_action:014b}'[1+(i//9)+1:], 2) # 駒の種類の反映
      set_action = int(f'{set_action:014b}'[:1+4+(i%9)] + '1' + f'{set_action:014b}'[1+4+(i%9)+1:], 2) # 駒の位置の反映
      set_actions.append(set_action)

  return set_actions

確認

In [81]:
# 確認

# choiseしている駒と同じ駒は行動できる。

# 先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
state = 0b_1_1_0_100000_000000000_000000000_000000000_000000000_000000000_01111111
print("先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')
# 先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
state = 0b_1_1_0_010000_000000000_000000000_000000000_000000000_000000000_000000000_000000000_11011111
print("先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')

# 後手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
state = 0b_0_1_0_0001_000000000_000000000_000000000_000000000_000000000_11111101
print("後手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')
print()

# choiseしている駒がboardの駒なら、そのマスに行動出来ない

# 先手, マス0の自駒Lを選択時, 行動位置は1~8のハズ(盤面に他の駒なし)
state = 0b_1_1_1_1000_100000000_000000000_000000000_000000000_000000000_01111111
print("先手, マス0の自駒Lを選択時, 置ける駒はL, その位置は1~8のハズ(盤面に他の駒なし)")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')
# 先手, マス3の自駒Mを選択時, 行動位置は0~2, 4~8のハズ(盤面に他の駒なし)
state = 0b_1_1_1_0100_000100000_000000000_000000000_000000000_000000000_11011111
print("先手, マス3の自駒Mを選択時, 置ける駒はM, その位置は0~2, 4~8のハズ(盤面に他の駒なし)")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')
print()

# choiseした駒以上の大きさの駒があるマスには行動できない。

# 先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, 0, BL, BM, 0], 行動位置は0~2, 4~5, 7~8のハズ
state = 0b_1_1_0_1000_000000000_000100000_000010000_000000100_000000010_00010101
print("先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, 0, BL, BM, 0], 行動位置は0~2, 4~5, 7~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')
# 先手, 手駒Mを選択時, 盤面には[0, 0, 0, RL, RM, 0, BL, BM, 0], 行動位置は0~2, 5, 8のハズ
state = 0b_1_1_0_0100_000000000_000100000_000010000_000000100_000000010_01000101
print("先手, 手駒Mを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 5, 8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')

先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
11000100000000
11000010000000
11000001000000
11000000100000
11000000010000
11000000001000
11000000000100
11000000000010
11000000000001
先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
10100100000000
10100010000000
10100001000000
10100000100000
10100000010000
10100000001000
10100000000100
10100000000010
10100000000001
後手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
10001100000000
10001010000000
10001001000000
10001000100000
10001000010000
10001000001000
10001000000100
10001000000010
10001000000001

先手, マス0の自駒Lを選択時, 置ける駒はL, その位置は1~8のハズ(盤面に他の駒なし)
11000010000000
11000001000000
11000000100000
11000000010000
11000000001000
11000000000100
11000000000010
11000000000001
先手, マス3の自駒Mを選択時, 置ける駒はM, その位置は0~2, 4~8のハズ(盤面に他の駒なし)
10100100000000
10100010000000
10100001000000
10100000010000
10100000001000
10100000000100
10100000000010
10100000000001

先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, 0, BL, BM, 0], 行動位置は0~2, 4~5, 7~8のハズ
11000100000000
11000010000000
11000001000000
11000000010000
11000000001000
11000000

#### leagal_actions_from_set_state(set_state) -> choise_actions:  

choiseできる駒の一覧... choise_actions を作成する  
choiseできる駒とは....  
- 自分の駒しか行動できない

手駒なら,
- 手駒に動かしたい駒が１つもなければ、その駒は行動できない。
- 動かしたい駒以上の大きさの駒が配置されているマスには行動できない。

盤上の駒なら、
- 盤上の一番表面にある駒しか行動できない。
- 動かしたい駒以上の大きさの駒があるマスには行動できない。

In [82]:
# set_stateから合法手を作成
#
# 出力するactionはすべてchoise_actionである。

def leagal_actions_from_set_state(set_state):
  is_first = f'{set_state:060b}'[0]
  board = f'{set_state:060b}'[16:16+36]
  hand = f'{set_state:060b}'[52:52+8]

  # 盤面の各マスに駒が存在するかどうか
  exist_place = f'{int(board[0:9], 2) | int(board[9:18], 2) | int(board[18:27], 2) | int(board[27:36], 2) :09b}' 
  # 手駒は,
  choise_hand_actions_bit = '1111'
  for piece_type in range(4): # 手駒にその駒が１つもないなら行動不可
    if hand[piece_type*2:piece_type*2+2] == "00": 
      choise_hand_actions_bit = choise_hand_actions_bit[:piece_type] + '0' + choise_hand_actions_bit[piece_type+1:]
  if is_first == "1": # 先手なら、先手の駒しか動かせない
    choise_hand_actions_bit = choise_hand_actions_bit[:2] + '00'
  else: # 後手なら、後手の駒しか動かせない
    choise_hand_actions_bit = '00' + choise_hand_actions_bit[2:]
  # 盤上の駒は、
  exist_large_place = int(board[0:9], 2) | int(board[18:27], 2)
  exist_middle_place = int(board[9:18], 2) | int(board[27:36], 2)
  # 盤上の一番表面にある駒しか行動できない
  choise_board_actions_bit = board[0:9] \
   + f'{int(board[9:18], 2) & ~exist_large_place :09b}' \
   + board[18:27] \
   + f'{int(board[27:36], 2) & ~exist_large_place :09b}'
  if is_first == "1": # 先手なら、先手の駒しか動かせない
    choise_board_actions_bit = choise_board_actions_bit[:18] + '000000000000000000'
  else: # 後手なら、後手の駒しか動かせない
    choise_board_actions_bit = '000000000000000000' + choise_board_actions_bit[18:]

  # choise_hand_actions_bitをchoise_actionsへchoise_actionに変換しながら追加
  choise_actions = []
  for i, bit in enumerate(choise_hand_actions_bit): # choise_hand_actions_bit = '00_00
    if bit == '1':
      choise_action = int('0b_0_0000_000000000', 2)
      choise_action = int(f'{choise_action:014b}'[:1+i] + '1' + f'{choise_action:014b}'[1+i+1:], 2) # 駒の種類の反映
      choise_actions.append(choise_action)

  # choise_board_actions_bitをchoise_actionsへchoise_actionに変換しながら追加
  for i, bit in enumerate(choise_board_actions_bit): # choise_board_actions_bit = '000000000_000000000_000000000_000000000'
    if bit == '1':
      choise_action = int('0b_1_0000_000000000', 2)
      choise_action = int(f'{choise_action:014b}'[:1+(i//9)] + '1' + f'{choise_action:014b}'[1+(i//9)+1:], 2) # 駒の種類の反映
      choise_action = int(f'{choise_action:014b}'[:1+4+(i%9)] + '1' + f'{choise_action:014b}'[1+4+(i%9)+1:], 2) # 駒の位置の反映
      choise_actions.append(choise_action)

  return choise_actions

確認

In [83]:
# 確認

# 手駒
# 先手, 手駒にその駒が１つもないなら行動不可
state = 0b_1_0_0_0000_000000000_000000000_000000000_000000000_000000000_00010000
print("# 先手, 手駒にその駒が１つもないなら行動不可。(行動可能は先手の駒数が1つのM駒のハズ)")
for action in leagal_actions_from_set_state(state):
  print(f'{action:014b}')
print()

# 盤上の駒
# 先手、盤上の一番表面にある駒しか行動できない。(行動可能は0,1マスにあるL駒と2マスにあるM駒)
state = 0b_1_0_0_0000_000000000_110000000_011000000_000000000_000000000_00001111
print("# 先手、盤上の一番表面にある駒しか行動できない。(行動可能は0マスにあるL駒と1マスにあるM駒)")
for action in leagal_actions_from_set_state(state):
  print(f'{action:014b}')

# 先手, 手駒にその駒が１つもないなら行動不可。(行動可能は先手の駒数が1つのM駒のハズ)
00100000000000

# 先手、盤上の一番表面にある駒しか行動できない。(行動可能は0マスにあるL駒と1マスにあるM駒)
11000100000000
11000010000000
10100001000000


#### 合法手の作成

In [84]:
def legal_actions(state):
  actions = []
  if f'{state:060b}'[1] == '1':
    actions = leagal_actions_from_choise_state(state)
  else:
    actions = leagal_actions_from_set_state(state)
  return actions

確認

In [85]:
# 確認

# 先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
state = 0b_1_1_0_1000_000000000_000000000_000000000_000000000_000000000_01111111
print("先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ")
for action in legal_actions(state):
  print(f'{action:014b}')

先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
11000100000000
11000010000000
11000001000000
11000000100000
11000000010000
11000000001000
11000000000100
11000000000010
11000000000001


### 次の手

stateにactionを反映して、次のstateを作成する

stateごとに関数を作成。  
- choise_stateから次のstateを作成 : next_state_from_choise_state(choise_state, set_action)
- set_stateから次のstateを作成 : next_state_from_set_state(set_state, choise_action)


#### choise_state, set_actionから次のstateを作成

In [86]:
def next_state_from_choise_state(choise_state, set_action):   
  is_first = f'{choise_state:060b}'[0]
  is_choise_board = f'{choise_state:060b}'[2]
  choise_piece_type = f'{choise_state:060b}'[3:3+4]
  choise_piece = f'{choise_state:060b}'[7:7+9]
  board = f'{choise_state:060b}'[16:16+36]
  hand = f'{choise_state:060b}'[52:52+8]

	# set_actionで選択している駒と位置を、boardで1にする。
  piece_type = f'{set_action:014b}'[1:5].find('1')
  piece_place = f'{set_action:014b}'[5:].find('1')
  board = board[:piece_type*9+piece_place] + '1' +  board[piece_type*9+piece_place+1:] 

	# 手番プレイヤーの交代 … is_firstを反転
  is_first = "0" if is_first == "1" else "1"
	# stateのモードを切り替える...is_choiseを0にする
  is_choise = "0"
  # is_choise_board, choise_piece_type, choise_piece をすべて0埋め
  is_choise_board = "0"
  choise_piece_type = "0000"
  choise_piece = "000000000"

  # next_stateの作成
  next_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return next_state

確認

In [87]:
# 確認

# (state = 先手初期浮き手駒Lのchoise盤面, action = Lを0マスに置く) => (state = 後手、先手L駒が0に配置された盤面のset状態)
state = 0b_1_1_0_1000_000000000_000000000000000000000000000000000000_01111111
action = 0b_1_1000_100000000
next_state = next_state_from_choise_state(state, action)
print("正解 : next_state = 000000000000000010000000000000000000000000000000000001111111")
print( f'結果 : next_state = {next_state:060b}')
print()

# (state = 後手, 先手Lが0かつ浮き手駒Lのchoise盤面, action = Lを1マスに置く) => (state = 先手、先手L駒が0, 後手L駒が1に配置された盤面のset状態)
state = 0b_0_1_0_0010_000000000_100000000000000000000000000000000000_01110111
action = 0b_1_0010_010000000
next_state = next_state_from_choise_state(state, action)
print("正解 : next_state = 100000000000000010000000000000000001000000000000000001110111")
print( f'結果 : next_state = {next_state:060b}')
print()

# (state = 先手, 先手L0後手L1浮き盤駒Lのchoise盤面, action = Lを2マスに置く) => (state = 後手、先手L駒が2、後手L駒が1に配置された盤面のset状態)
state = 0b_1_1_1_1000_100000000_000000000000000000010000000000000000_01110111
action = 0b_1_1000_001000000
next_state = next_state_from_choise_state(state, action)
print("正解 : next_state = 000000000000000000100000000000000001000000000000000001110111")
print( f'結果 : next_state = {next_state:060b}')
print()

正解 : next_state = 000000000000000010000000000000000000000000000000000001111111
結果 : next_state = 000000000000000010000000000000000000000000000000000001111111

正解 : next_state = 100000000000000010000000000000000001000000000000000001110111
結果 : next_state = 100000000000000010000000000000000001000000000000000001110111

正解 : next_state = 000000000000000000100000000000000001000000000000000001110111
結果 : next_state = 000000000000000000100000000000000001000000000000000001110111



#### set_state, choise_actionから次のstateを作成

In [88]:
def next_state_from_set_state(set_state, choise_action):
  is_first = f'{set_state:060b}'[0]
  is_choise_board = f'{set_state:060b}'[2]
  choise_piece_type = f'{set_state:060b}'[3:3+4]
  choise_piece = f'{set_state:060b}'[7:7+9]
  board = f'{set_state:060b}'[16:16+36]
  hand = f'{set_state:060b}'[52:52+8]

	# stateのモードを切り替える...is_choiseを1にする
  is_choise = "1"
  # choise_actionをis_choise_board, choise_piece_type, choise_piece に反映する
  is_choise_board = f'{choise_action:014b}'[0]
  choise_piece_type = f'{choise_action:014b}'[1:5]
  choise_piece =  f'{choise_action:014b}'[5:14]
  # 選択した駒をboard, handから取り除く
  piece_type = f'{choise_action:014b}'[1:5].find('1')
  piece_place = f'{choise_action:014b}'[5:].find('1')
  if f'{choise_action:014b}'[0] == "1":   # choise_actionで盤の駒を選択しているなら
    # choise_actionで選択している駒と位置を、boardで0にする。
    board = board[:piece_type*9+piece_place] + '0' +  board[piece_type*9+piece_place+1:] 
  # choise_actionで選択している駒の数を、handで1減らす。
  # “00”...その駒は持ってない, “01”...一つ持ってる, “11”...２つ持ってる	
  if hand[piece_type*2:piece_type*2+2] == "11":
    hand = hand[:piece_type*2] + "01" + hand[piece_type*2+2:]
  else: # hand[piece_type*2:piece_type*2+2] == "01":
    hand = hand[:piece_type*2] + "00" + hand[piece_type*2+2:]

  # next_stateの作成
  next_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return next_state

In [89]:
def create_next_state(state, action):
  is_choise = f'{state:060b}'[1]
  next_state = 0
  if is_choise == "1":
    next_state = next_state_from_choise_state(state, action)
  else:
    next_state = next_state_from_set_state(state, action)
  next_state = convert_normalization_state(next_state) # 下記確認ではコメントアウト
  return next_state

#### 確認

actions が空になるまで、初期盤面からnext_stateを探索していく。  
ここでは、`action = actions[0]`とする。

In [90]:
# 確認
state = 0b_1_0_0_0000_000000000_000000000000000000000000000000000000_11111111
print("0番目")
print("状態 : {}".format(state))
action = legal_actions(state)[0]
print("行動 : {}".format(action))
for i in range(1,10):
  print(str(i) + "番目")
  state = create_next_state(state, action)
  print("状態 : {}".format(state))
  action = legal_actions(state)[0]
  print("行動 : {}".format(action))
  print()


0番目
状態 : 576460752303423743
行動 : 4096
1番目
状態 : 936748722493063295
行動 : 12544

2番目
状態 : 34359738495
行動 : 1024

3番目
状態 : 306244809020932215
行動 : 9472

4番目
状態 : 576460786696716407
行動 : 4096

5番目
状態 : 936748756886356023
行動 : 12416

6番目
状態 : 412325249079
行動 : 1024

7番目
状態 : 306245186986442803
行動 : 9472

8番目
状態 : 576461164662226995
行動 : 2048

9番目
状態 : 900720337832902675
行動 : 10368



### 盤面から勝敗確定かチェック

盤面に対して、lineがあるかどうか、あるならどちらの駒かを確認する。
```
def check_line(state) -> [is_line, is_first_line]:
	stateのboard部分からsurface_boardを作成
	# プレイヤーごとに３目並べの判定をし、もし揃っていたら
		is_line = True
		is_first_line = そのlineがfirst_playerのものかどうか
	# そろってなかったら, is_lineはFalse
	return [is_line, is_first_line]
```

surface_board
 … boardで上に駒が重なっている駒を0にした。

追記: check_result(state) -> [is_done, winner]:に変更  
- is_done : lineがあるか
- winner : 先手なら0, 後手なら1

In [130]:
# 勝敗の有無

check_lines = np.array([[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [2,4,6]])

def is_win(single_surface):
  s = single_surface
  # 横, 縦, 左斜め, 右斜めのラインを調べる
  for check_line in check_lines:
    if s[check_line[0]] and s[check_line[1]] and s[check_line[2]]:
      return True;
  return False;

In [131]:
# 確認
single_surface = np.array([0,0,0,0,0,0,0,0,0])
print(is_win(single_surface))
single_surface = np.array([1,0,0,0,0,0,0,0,0])
print(is_win(single_surface))
single_surface = np.array([1,1,0,0,0,0,0,0,0])
print(is_win(single_surface))
single_surface = np.array([1,1,1,0,0,0,0,0,0])
print(is_win(single_surface))

False
False
False
True


In [132]:
# 表面の駒だけbitが立つboardを作成する
def create_surface(board):
  # boardの変換
  board_list_list = [0] * 4
  for i in range(4):
    board_list_list[i] = list(board[i*9:i*9+9])
  board_list_list = np.array(board_list_list)
  board_surface = np.array(
      [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]])
  # M駒を反映させる
  board_surface[1] = np.array([int(i_str) for i_str in board_list_list[1]])
  board_surface[3] = np.array([int(i_str) for i_str in board_list_list[3]])

  # このとき、おいた場所により小さい駒があったら、その駒を0にする
  # １マスずつ見ていく
  for place in range(9):
    # L駒を反映させる
    if board_list_list[0][place] == "1" or board_list_list[2][place] == "1":
      if board_list_list[0][place] == "1":
        board_surface[0][place] = 1
      if board_list_list[2][place] == "1":
        board_surface[2][place] = 1
      board_surface[1][place] = 0 # M駒を0にする
      board_surface[3][place] = 0
  return board_surface

In [133]:
# 確認
board = "000000100001000100011000000000011111"
print(create_surface(board))

[[0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 1 1]]


In [134]:
#勝敗の有無、勝者を確認する
def check_result(state):
  board = f'{state:060b}'[16:16+36]
  board_surface = create_surface(board)
  is_done = 0 # 決着がついているなら1を返す
  winner = 0 # 先手は0, 後手は1
  # 内部でboard_surface[0,1]とboard_surface[2,3]を合成
  single_surfaces = np.array([[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]])
  for i in range(9):
    if board_surface[0][i] == 1 or board_surface[1][i] == 1:
      single_surfaces[0][i] = 1
    elif board_surface[2][i] == 1 or board_surface[3][i] == 1:
      single_surfaces[1][i] = 1
  if is_win(single_surfaces[0]) == 1 or  is_win(single_surfaces[1]) == 1:
    is_done = 1
  winner = 1 if is_win(single_surfaces[1]) == 1 else 0
  return [is_done, winner]

In [140]:
# 確認
state = 0b_1_0_0_0000_000000000_000000000000000000000000000000000000_11111111
is_done, winner = check_result(state)
print(is_done, winner)
# 確認
state = 0b_1_0_0_0000_000000000_110000000001000000000000000000000000_00011111
is_done, winner = check_result(state)
print(is_done, winner)
state = 0b_0_0_0_0000_000000000_000000000000000000110000000001000000_11110001
is_done, winner = check_result(state)
print(is_done, winner)

0 0
1 0
1 1


## 全状態の列挙

In [141]:
import time
from collections import deque
import sys
from IPython.display import clear_output

In [142]:
%%time

# 到達可能な全盤面を作成する
# 発見 -> 未訪問 -> 訪問済み の流れ。

q = deque() # 発見キュー
all_states = set() # 訪問済み

# 初期状態の追加
init_state = 0b_1_0_0_0000_000000000_000000000000000000000000000000000000_11111111
q.append(convert_normalization_state(init_state))
# qが空になるまで探索
cnt = 0
while q: 
  cnt += 1
  if cnt % 100 == 0:
    clear_output(wait=True)
    print(cnt)
    time.sleep(0.001) #1msの休止
  current_state = q.popleft()
  if current_state not in all_states: # current_stateがまだ訪問済みでないなら
    all_states.add(current_state) # 訪問済みに追加
    is_done, winner = check_result(current_state)
    if not is_done: # 勝敗決定盤面でないなら
      for action in legal_actions(current_state): # 合法手を列挙
        q.append(create_next_state(current_state, action)) #次の状態(正規化済)をqに追加
        
# list化してソート
allstates = list(all_states)
allstates.sort()

# 状態の確認
clear_output(wait=True)
print(f'状態数: {len(allstates)}')
print(f'サイズ: {sys.getsizeof(allstates)} byte')
print(f'loop回数: {cnt}')

2500


KeyboardInterrupt: 

In [96]:
# pickleで保存
import pickle
with open('allstates.pickle', 'wb') as f:
    pickle.dump(allstates, f)

NameError: name 'allstates' is not defined

## 全状態の合法手を列挙

In [ ]:
# allstatesを非pickle化
with open('allstates.pickle', 'rb') as f:
    allstates = pickle.load(f)

# allLegalActionsを全状態数分用意する
# allLegalActions[i]... allstates[i]の合法手一覧
allLegalActions = [[]] * len(allstates)
print(len(allstates))

In [225]:
# allLegalActionsの作成

for i, state in enumerate(tqdm(allstates, desc="合法手を計算", leave=True)):
  is_done, winner = check_result(state)
  if is_done: # 終了盤面    
    allLegalActions[i] = legal_actions(state)
  else:
      pass; # []のまま


In [226]:
# pickleで保存
with open('allLegalActions.pickle', 'wb') as f:
    pickle.dump(allLegalActions, f)

## 全状態の次の状態を列挙

In [232]:
# allstates, allLegalActionsを非pickle化
with open('allstates.pickle', 'rb') as f:
    allstates = pickle.load(f)
with open('allLegalActions.pickle', 'rb') as f:
    allLegalActions = pickle.load(f)

# allNextStatesを全状態数分用意する
# allNextStates[i]... allstates[i]の次の状態一覧
allNextStates = [[]] * len(allstates)
print(len(allNextStates))

5116724


In [233]:
# allNextStatesの作成
for i, state in enumerate(tqdm(allstates, desc="allNextStatesの作成", leave=True)):
  is_done, winner = check_result(state)
  if not is_done: # 終了盤面
    next_states = []
    for action in allLegalActions[i]: # 合法手をallLegalActionsから取ってくる
      next_state = create_next_state(state, action)
      next_states.append(next_state)
    allNextStates[i] = next_states
  else:
    pass; # 末端は[]のまま

In [234]:
# pickleで保存
with open('allNextStates.pickle', 'wb') as f:
    pickle.dump(allNextStates, f)

## 後退解析

### 追加関数

In [210]:
# # 過去
# # 勝敗判定を行う
# # 勝ち色を返す (1...R, -1...B, 未確定なら0)
# # 組み込んだ
# def newWinLose(arg_allstates, arg_winLose, arg_state):
#   is_first = int(f'{arg_state:060b}'[0])
#   turn_color = 1 if is_first == 1 else -1
#   alllose = True;
#   for action in legal_actions(state): # 合法手を列挙
#     next_state = create_next_state(arg_state, action)
#     i1 = arg_allstates.index(next_state)
#     if arg_winLose[i1] == turn_color: # 勝ち盤面が１つでもあればそれを指せば勝ち
#       return turn_color;
#     if arg_winLose[i1] == 0:
#       alllose = False;
#   if alllose: # 次の手がすべて相手の勝ち盤面なら相手の勝ち
#     return -turn_color;
#   else:
#     return 0;

In [244]:
# allstates, allLegalActions, allNextStatesをを用いて勝敗判定を行う

# allstates, allLegalActions, allNextStatesを非pickle化
with open('allstates.pickle', 'rb') as f:
    allstates = pickle.load(f)
with open('allLegalActions.pickle', 'rb') as f:
    allLegalActions = pickle.load(f)
with open('allNextStates.pickle', 'wb') as f:
    pickle.dump(allNextStates, f)

def newWinLose(arg_allstates, arg_winLose, arg_state, arg_allNextStates):
  is_first = int(f'{arg_state:060b}'[0])
  turn_color = 1 if is_first == 1 else -1
  alllose = True;
  i = arg_allstates.index(arg_state)
  for next_state in arg_allNextStates[i]:
    i1 = arg_allstates.index(next_state)
    if arg_winLose[i1] == turn_color: # 勝ち盤面が１つでもあればそれを指せば勝ち
      return turn_color;
    if arg_winLose[i1] == 0:
      alllose = False;
  if alllose: # 次の手がすべて相手の勝ち盤面なら相手の勝ち
    return -turn_color;
  else:
    return 0;

### 処理

In [247]:
# allstates, allLegalActions, allNextStatesを非pickle化
with open('allstates.pickle', 'rb') as f:
    allstates = pickle.load(f)
with open('allLegalActions.pickle', 'rb') as f:
    allLegalActions = pickle.load(f)
with open('allNextStates.pickle', 'wb') as f:
    pickle.dump(allNextStates, f)

# winLose, winLoseCountを全状態数分用意する
# winLose[i]... allstates[i]の勝敗. 未確定が0, R勝ちが1, B勝ちが-1
# winLoseCount ... allstates[i]が決着までかかる手数
winLose = [0] * len(allstates)  # その盤面の勝敗 0は未確定, 1はR, -1はB
winLoseCount = [0] * len(allstates) # 勝敗にかかるまでの手数
count= {"Redwin": 0, "Bluewin" : 0, "yet" : 0}
print(len(allstates))


5116724


In [206]:
# 進行バー
from tqdm.notebook import tqdm as tqdm
for i in tqdm(range(100), desc="進捗", leave=True ):
  time.sleep(0.1)

In [ ]:
%%time

# 現在の盤面のwinLoseを記録
for i, state in enumerate(tqdm(allstates, desc="初回の勝敗判定", leave=True)):
  is_done, winner = check_result(state)
  if is_done: # 終了盤面
    if winner == 0: #FIRST_PLAYER RED
      winLose[i]=1;
      count["Redwin"] += 1;
    elif winner == 1: #SECOND_PLAYER BLUE
      winLose[i]=-1;
      count["Bluewin"] += 1;
  else:
      winLose[i]=0;
      count["yet"] += 1;

print("後退解析開始")
c = 0 # 手数
while True:
  # 勝敗のカウント
  c += 1;
  print(f'iteration {c}')
  print(f'Redwin : {count["Redwin"]}, Bluewin : {count["Bluewin"]}, yet : {count["yet"]}')
  # 解析終了フラグ
  changed = False 
  # 解析
  for i, state in enumerate(tqdm(allstates, desc=f'iteration {c}', leave=True)):
    if winLose[i] == 0: # 未決着の盤面のみ判定する
      nv = newWinLose(allstates, winLose, state, allNextStates) # 勝敗
      if nv != 0:
        winLose[i] = nv # この状態の理論値を入力
        winLoseCount[i] = c # 勝敗にかかるまでの手数を入力
        if nv == 1:
          count["Redwin"] += 1;#決着の盤面数を増やす
        elif nv == -1:
          count["Bluewin"] += 1;
        count["yet"] -= 1; # 未決着盤面数をへらす
        changed = True # 解析継続
  if not changed:
    break;

print("解析終了")
print(f'Redwin : {count["Redwin"]}')
print(f'Bluewin : {count["Bluewin"]}')
print(f'あいこ : {count["yet"]}')
print(f'ループ回数 : {c}')


後退解析開始
iteration 1
Redwin : 72153, Bluewin : 192818, yet : 4851753


In [189]:
# winLose, winLoseCountをpickle化
# import pickle
# with open('winLose.pickle', 'wb') as f:
#     pickle.dump(winLose, f)
# with open('winLoseCount.pickle', 'wb') as f:
#     pickle.dump(winLoseCount, f)
